In [9]:
import os 
import glob

def load_files_from_directory(input_dir="../graphragtest/input"):
    """
    Load all files from the specified directory and return their combined content.
    
    Args:
        input_dir (str): Path to the directory containing files to load
        
    Returns:
        tuple: (files_list, combined_content)
            - files_list: List of file paths found
            - combined_content: String containing all file contents concatenated
    """
    files = []
    
    # Get all files in the directory
    for file_path in glob.glob(os.path.join(input_dir, "*")):
        if os.path.isfile(file_path):
            files.append(file_path)
    
    print(f"Found {len(files)} files:")
    for file in files:
        print(f"  {file}")
    
    # Load content from all files
    content = ''
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            file_content = f.read()
            content += file_content
    
    print(f"Loaded content from {len(files)} file(s)")
    
    return files, content

In [10]:
files, content = load_files_from_directory()

Found 1 files:
  ../graphragtest/input\GOT_chapter_summaries.txt
Loaded content from 1 file(s)


In [11]:
content

'A Game of Thrones - Prologue\nPlace(s): \nHaunted forest\n \n \nNavigation:\nPrologue\n→ \nBran I\nThree \nrangers\n of the \nNight\'s Watch\n are tracking a group of \nwildling\n raiders \nbeyond the Wall\n, and encounter the\nOthers\n, who had not been seen for eight thousand years.\nSynopsis\nGared\n, \nWill\n, and \nSer\n \nWaymar Royce\n, three \nrangers\n from the \nNight\'s Watch\n, are tracking a band of \nwildling\n raiders\nin the \nhaunted forest\n. Will has reported on wildlings that he found, claiming they are all dead. Gared, an older man\nwho has been in the Night\'s Watch for decades, is uneasy, and insists that they turn back to the \nWall\n; they have eight\nor nine days of travel ahead of them, which can turn into a fortnight if it snows. Waymar, a noble born youth of\neighteen, however, has the command, and after making light of Gared\'s fears asks Will again for the details of what\nhe saw.\nWill explains that he saw the wildlings\' encampment. Their lean-to was c

In [12]:
from dotenv import load_dotenv
import os
load_dotenv()

API_KEY = os.getenv('GRAPHRAG_API_KEY')
print(f"API Key loaded: {'Yes' if API_KEY else 'No'}")

API Key loaded: Yes


In [13]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from langchain_mistralai import MistralAIEmbeddings
from time import sleep

embeddings = MistralAIEmbeddings(
    model="mistral-embed",
    api_key=API_KEY,
)

class MistralEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        # Handle both single string and list of strings
        sleep(1)
        if isinstance(input, str):
            return [embeddings.embed_query(input)]
        elif isinstance(input, list):
            return [embeddings.embed_query(doc) for doc in input]
        else:
            raise ValueError("Input must be a string or list of strings")
    
    def name(self) -> str:
        return "mistral-embed"

In [ ]:
import chromadb

chroma_client = chromadb.PersistentClient(path="../rag/chromadb")

# Delete existing collection if it exists to avoid embedding function mismatch
# try:
#     chroma_client.delete_collection(name="collection")
#     print("Deleted existing collection")
# except:
#     print("No existing collection to delete")

collection = chroma_client.get_or_create_collection(
    name="collection", 
    embedding_function=MistralEmbeddingFunction()
)
print("Created new collection with Mistral embeddings")

Deleted existing collection
Created new collection with Mistral embeddings


C:\Users\joaow\AppData\Local\Temp\ipykernel_14136\715117250.py:14: DeprecationWarning: The class MistralEmbeddingFunction does not implement __init__. This will be required in a future version.
  embedding_function=MistralEmbeddingFunction()


In [15]:
from langchain_text_splitters import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=100)

texts = text_splitter.split_text(content)

In [16]:
len(texts)

447

In [17]:
for index, text in enumerate(texts):
    print(f"Adding chunk {index} to collection")
    collection.add(
        documents=[text],
        ids=f"chunk_{index}"
    )

Adding chunk 0 to collection
Adding chunk 1 to collection
Adding chunk 2 to collection
Adding chunk 3 to collection
Adding chunk 4 to collection
Adding chunk 5 to collection
Adding chunk 6 to collection
Adding chunk 7 to collection
Adding chunk 8 to collection
Adding chunk 9 to collection
Adding chunk 10 to collection
Adding chunk 11 to collection
Adding chunk 12 to collection
Adding chunk 13 to collection
Adding chunk 14 to collection
Adding chunk 15 to collection
Adding chunk 16 to collection
Adding chunk 17 to collection
Adding chunk 18 to collection
Adding chunk 19 to collection
Adding chunk 20 to collection
Adding chunk 21 to collection
Adding chunk 22 to collection
Adding chunk 23 to collection
Adding chunk 24 to collection
Adding chunk 25 to collection
Adding chunk 26 to collection
Adding chunk 27 to collection
Adding chunk 28 to collection
Adding chunk 29 to collection
Adding chunk 30 to collection
Adding chunk 31 to collection
Adding chunk 32 to collection
Adding chunk 33 to c

In [18]:
def retrieval(query, num_results=5):
    results = collection.query(
        query_texts=[query],
        n_results=num_results
    )
    return results

In [19]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("mistral-medium-latest", model_provider="mistralai", temperature=0, api_key = API_KEY)

In [32]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

rag_prompt_template = """
Generate a response of the target length and format that responds to the user's question, summarizing all information in the input data tables appropriate for the response length and format

If you don't know the answer, just say so. Do not make anything up.

Do not include information where the supporting evidence for it is not provided.
Do not answer unless its totally specified in the context.
Your only source of facts is the provided context.

Context: {retrieved_docs}

User Question: {query}

"""

rag_prompt = ChatPromptTemplate.from_template(rag_prompt_template)

rag_chain = rag_prompt | llm | StrOutputParser()

In [21]:
def chroma_rag(query):
    retrieved_docs = retrieval(query)["documents"][0]
    print(retrieved_docs)
    response = rag_chain.invoke({"retrieved_docs": retrieved_docs, "query": query})
    return response

In [22]:
from IPython.display import display, Markdown

In [23]:
display(Markdown(chroma_rag('How is Daenerys Targaryen related to the old man from the Night\'s Watch?')))

[' the northern side. Many of the men of the Night\'s Watch are also muttering against the\nperception that Jon has taken Stannis\' side in the game of thrones. Jon denies this, but says that the outcome of the\nstruggle is unclear following the death of \nTywin Lannister\n. Jon shares a cup of wine with \nClydas\n, telling him that\nMaester \nAemon\n had shown him a passage in the \nJade Compendium\n describing \nLightbringer\n as both bright and\nfearfully hot. He notes that although Stannis\'s sword radiates light, it produces no heat.\nNavigation:\nJon II\n↑\nDavos I\n ←\nJon III\n→ \nDaenerys II\n↓\nJon IV\n\nA Dance with Dragons - Chapter 11: Daenerys II\nPlace(s): \nMeereen\n \n \nNavigation:\nDaenerys II\n↑\nJon III\n ←\nDaenerys II\n→ \nReek I\n↓\nDaenerys III\nSynopsis\nThere have been more murders by the \nHarpy\'s Sons\n on the streets of \nMeereen\n. This time they involved an attack on\nfully-armed \nUnsullied\n in pairs. Nine died, including one of \nMissandei\n\'s broth

Based on the provided context, there is no direct information about how Daenerys Targaryen is related to the old man from the Night's Watch. The context does not specify any familial or other connections between them.

If you have more details or a different question, feel free to ask!

In [24]:
display(Markdown(chroma_rag("Is it possible that Jon Snow is the son of Lyanna Stark and Rhaegar Targaryen? Does that make him the azor ahai?")))

[' I\n↓\nJon II\nSynopsis\nIn \nJon\n\'s dream he sees through the eyes of his direwolf, \nGhost\n, who is hunting in the woods somewhere south of the\nWall. In the dream Ghost is aware of the presence of the other stark direwolves \nNymeria\n, \nShaggydog\n and \nSummer\n.\nJon is awakened from the dream by \nJeor Mormont\n\'s raven. He resides in the old quarters of \nDonal Noye\n, since\nStannis\n occupies the King’s Tower and the Lord Commander’s Tower has burned down. He reflects that Ghost\nknows that \nGrey Wind\n is dead, but his other brothers\' wolves are still alive. His wolf dreams are becoming more\nvivid and more frequent, and he wonders if part of \nBran\n and \nRickon\n survive in their wolves.\nAs \nLord Commander of the Night\'s Watch\n, Jon heads off to treat with King Stannis, who has requested his presence.\nAs Jon crosses the yard, he gives advice to sparring recruits and is challenged to a duel by Ser \nGodry Farring\n, from\nStannis’ retinue, but ignores the cha

Based on the provided context, there is no direct evidence or confirmation that Jon Snow is the son of Lyanna Stark and Rhaegar Targaryen. The context does not explicitly address Jon's parentage or any connection to the Azor Ahai prophecy.

The context does mention:
- Jon Snow's dreams and experiences with his direwolf, Ghost.
- His interactions with Stannis, Melisandre, and other characters.
- References to Targaryen history, including Maester Aemon's lineage and the Azor Ahai prophecy, but no direct link to Jon.

Without explicit information in the context, I cannot confirm or deny Jon Snow's parentage or his potential role as Azor Ahai.

In [25]:
display(Markdown(chroma_rag('What were the key events that led to the downfall of House Stark, and how did the actions of key characters like Eddard Stark, Catelyn Stark, and Robb Stark contribute to this downfall?')))

[" She then introduces the\nManderly brothers and her uncle.\nWhen Robb asks, Catelyn explains that she has sent Ser \nRodrik\n back to \nWinterfell\n to act as \ncastellan\n and to assist\nMaester\n \nLuwin\n in military matters. After some of the lords give Catelyn reassurance that Winterfell is safe, Lord\nRoose Bolton\n states that it is rumored she holds \nTyrion Lannister\n, who would be a good hostage. Catelyn admits she\nlost the dwarf because the gods saw fit to free him with help from \nher sister\n. Speaking of Lysa, Catelyn recalls that\nbefore leaving the \nEyrie\n, she attempted to convince her sister to let her take \nRobert Arryn\n as a fosterling, but Lysa’s\nrage had been frightening.\nCatelyn then sends the bannermen and Theon away to speak to Robb alone. Once they are gone, Catelyn notes that\nher son looks older, especially with his new beard. Catelyn, who is afraid for her son, notes to Robb that he is only\nfifteen and leading a host to war. Robb insists that the

The downfall of House Stark was the result of a series of key events and decisions made by its members, particularly Eddard Stark, Catelyn Stark, and Robb Stark. Here are the pivotal moments and actions that contributed to their decline:

1. **Eddard Stark's Honor and Political Maneuvering**:
   - Eddard Stark's rigid sense of honor led him to reject Cersei Lannister's offer to flee King's Landing after discovering her secret about Joffrey's illegitimacy. Instead, he confronted her directly, which ultimately led to his arrest and execution.
   - His trust in Littlefinger and failure to secure immediate support from Renly Baratheon or other allies left him vulnerable to Cersei's machinations.

2. **Catelyn Stark's Arrest of Tyrion Lannister**:
   - Catelyn's decision to arrest Tyrion Lannister on her way to King's Landing, based on Lysa Arryn's letter accusing the Lannisters of Jon Arryn's murder, escalated tensions between House Stark and House Lannister.
   - This act provoked Tywin Lannister into sending forces into the Riverlands, sparking open warfare and drawing the Starks into a larger conflict.

3. **Robb Stark's Military Campaigns and Political Decisions**:
   - Robb's successful military campaigns initially bolstered House Stark's position, but his decision to be crowned "King in the North" alienated potential allies who might have supported him as a Stark loyal to the Iron Throne.
   - His failure to secure a strategic marriage alliance (breaking his promise to marry a Frey) led to the Red Wedding, where he, his mother, and many of his bannermen were betrayed and slaughtered by the Freys and Boltons.

4. **The Red Wedding**:
   - The betrayal at the Red Wedding, orchestrated by Tywin Lannister, Roose Bolton, and Walder Frey, was the final blow to House Stark. Robb, Catelyn, and much of their army were killed, effectively dismantling House Stark's military and political power.

5. **Bran and Rickon's Presumed Deaths**:
   - Theon Greyjoy's betrayal and the apparent deaths of Bran and Rickon Stark (later revealed to be a ruse) further weakened House Stark by removing the remaining heirs and destabilizing their hold on the North.

6. **Sansa and Arya's Captivity**:
   - Sansa's captivity in King's Landing and Arya's flight from the capital left the Stark children scattered and vulnerable, removing potential future leaders for the house.

These events, driven by a mix of honor, mistrust, and strategic missteps, culminated in the near-total destruction of House Stark. The actions of Eddard, Catelyn, and Robb, while often well-intentioned, ultimately played into the hands of their enemies, particularly the Lannisters.

In [26]:
display(Markdown(chroma_rag('Who ruled the Seven Kingdoms before and after Robert Baratheon? How did the kings died and what were the circumstances of their deaths?')))

["Robert Baratheon\n, strong as a bull,\nfearless in battle and loving war, who killed her brother \nRhaegar Targaryen\n, and his dogs: \nEddard Stark\n, \nTywin\nLannister\n and \nJaime Lannister\n. How will she overthrow them? Even her dragons might not be enough. Viserys, who\nwas a fool, believed the realm would rise for their rightful king. She then decides that the \nBleeding Star\n has led her\nto Qarth for a purpose, and if the gods intend for her to conquer they will send a sign.\nNear eveningfall, \nIrri\n brings word that Ser Jorah has returned. He enters with \nQuhuru Mo\n, captain of the \nCinnamon\nWind\n out of \nTall Trees Town\n. Daenerys has her dragons all around her. He has the news that Robert Baratheon is\ndead, reported in \nOldtown\n, \nDorne\n and \nLys\n. There are tales that \nthe queen\n, his brother (\nStannis Baratheon\n or \nRenly\nBaratheon\n, or \nEddard Stark\n betrayed him. \nJoffrey Baratheon\n sits on the throne with \nHouse Lannister\n ruling, Robe

Based on the provided context, here is the information about the rulers of the Seven Kingdoms before and after Robert Baratheon, along with the circumstances of their deaths:

### Before Robert Baratheon:
1. **Aerys II Targaryen (The Mad King)**
   - **Death**: Killed by Jaime Lannister during the Sack of King's Landing.
   - **Circumstances**: Aerys was overthrown during Robert's Rebellion. He was killed by Jaime Lannister, who was a member of the Kingsguard, to prevent Aerys from using wildfire to destroy the city.

### Robert Baratheon:
- **Death**: Mortally wounded by a boar while hunting.
- **Circumstances**: Robert was injured by a boar during a hunting trip. The wound became infected, and he died shortly afterward. His death led to a power struggle among his brothers and other claimants to the throne.

### After Robert Baratheon:
1. **Joffrey Baratheon**
   - **Death**: Poisoned at his own wedding feast.
   - **Circumstances**: Joffrey was poisoned during the Purple Wedding, an event orchestrated by various parties, including Olenna Tyrell and Littlefinger.

The context does not provide detailed information about other rulers who may have come after Joffrey, such as Tommen Baratheon or others. Therefore, the information is limited to Aerys II Targaryen, Robert Baratheon, and Joffrey Baratheon.

In [33]:
query = "Who are the main candidates for the prince that was promised? What are the main theories about this prophecy?"
display(Markdown(chroma_rag(query)))

[' all. He leads her to the\nentrance, warning her to always take the door to her right, and only take stairs leading up. On her way she will see\nvisions of the past, future and days that never were, and those that will speak to her from other rooms, but she must\nnot enter any room and keep her path until she reaches the Undying\'s audience chamber.\nAt the entrance, Daenerys drinks a glass of \nshade of the evening\n a dwarf gives her, to prepare her for the truths\nwithin, and enters. In the House of the Undying, she notices right away that she is in the presence of sorcery.\nFortunately, she brought \nDrogon\n with her into the Warlocks\' home. Daenerys sees a great many visions during her\njourney through the halls:\n1\n. \nA beautiful naked woman sprawling on a floor with four little men with rat-like faces raping her.\n2\n. \nA feast of corpses, a room full of savagely slaughtered bodies, and many of the corpses have cups or spoons in\ntheir hands; above them sits a dead man on

Based on the provided context, the main candidates for the "Prince That Was Promised" prophecy are:

1. **Aegon (Daenerys' nephew)** – In Daenerys' vision, a man resembling Viserys but with darker eyes refers to a baby named Aegon as "the Prince that was Promised" and mentions "the song of ice and fire."

2. **Daenerys Targaryen** – The prophecy mentions "the dragon has three heads," and Daenerys is referred to as the "Mother of Dragons." Some visions and the Undying's words suggest her role in the prophecy.

### Main Theories About the Prophecy:
- **"The Prince That Was Promised"** is a savior figure destined to battle the darkness (possibly the White Walkers).
- The prophecy may involve **three key figures** ("the dragon has three heads"), possibly Daenerys, Jon Snow, and another Targaryen (such as Aegon).
- The phrase **"the song of ice and fire"** suggests a union of opposing forces, possibly hinting at a Stark-Targaryen alliance.
- The Undying's cryptic words ("three fires must you light," "three treasons will you know") imply trials and choices tied to the prophecy.

The context does not provide definitive answers, but these are the key references made in the given passages.

In [35]:
query = "What are all the cities daenerys targaryen has visited? What are the main events that happened in each of these cities?"
display(Markdown(chroma_rag(query)))

[' grown now, and all because of Daenerys, the Mother of Dragons. Quaithe warns, "You must leave this\ncity soon…or you will never be permitted to leave it at all." Daenerys realizes that Quaithe wants her to go to\nAsshai\n, and when she asks why, the shadowbinder tells her it is there she will find truth. Her khas warn her not to\ntrust the Asshai’i shadowbinder, and Xaro concurs. Later, Ser \nJorah\n tells her she will find no help in \nthis city\n. He\nexplains why Xaro wants to marry her: During a Qartheen wedding ceremony, the bride and groom each ask one\nthing from each other that the other cannot refuse. Ser Jorah is convinced that Xaro plans to obtain one of her\ndragons in this way. He tells her she should go east, but not to Asshai. When Daenerys suggests returning west to\nIllyrio\n, Jorah warns her that the Magister cares only for himself and would provide no help. Daenerys believes he\nwill give her the ships and soldiers to retake \nWesteros\n, but Jorah states, "Nothin

Based on the provided context, here are the cities Daenerys Targaryen has visited and the main events that occurred in each:

1. **Vaes Dothrak**
   - Daenerys arrives with her khalasar and learns about the traditions and rules of the city, such as the prohibition of weapons and bloodshed.
   - She is informed that Khal Drogo must ascend the Mother of Mountains to sacrifice for his safe return.
   - Daenerys reflects on the Dothraki culture and the role of the dosh khaleen (widows of past khals).

2. **Qarth**
   - Daenerys seeks help and ships to retake Westeros but faces distrust and warnings from her advisors, including Ser Jorah Mormont and Xaro Xhoan Daxos.
   - She visits the House of the Undying, where she experiences visions of the past, future, and alternate realities.
   - Daenerys is warned by Quaithe to leave the city or risk being trapped there forever.

3. **Astapor**
   - Daenerys negotiates with the slavers to acquire the Unsullied, an army of slave soldiers.
   - She frees Missandei, a slave who speaks High Valyrian, and takes her into her service.
   - Daenerys commands Drogon to attack the slavers, leading to the conquest of Astapor and the liberation of the slaves.

These are the cities mentioned in the context along with the significant events that transpired in each.